<h1>Numpy and Pandas Home Assignment</h1>
In this assignment, we will use pandas and numpy to do some fairly basic analysis on equities.

<h3>Imports</h3>

In [4]:
!pip install --upgrade --quiet yfinance pandas-datareader

In [1]:
import datetime
import numpy as np
import pandas as pd
%matplotlib inline
import pandas_datareader.data as web
import yfinance as yf

<h3>Get data on the following four stocks from yahoo finance</h3>
<li>MS, GS, JPM, C
<li>Use the date range 2013/1/1/ to 2018/9/24

In [2]:
MS = yf.Ticker("MS").history(period="max")['2013-1-1':'2018-9-24']
GS = yf.Ticker("GS").history(period="max")['2013-1-1':'2018-9-24']
JPM = yf.Ticker("JPM").history(period="max")['2013-1-1':'2018-9-24']
C = yf.Ticker("C").history(period="max")['2013-1-1':'2018-9-24']

<h3>Compute the cross correlation coefficients on the stocks</h3>
<li>Create a df 'rets' with the one day percentage changes
<li>Calculate cross correlations on the rets df

In [3]:
rets = pd.concat([MS['Close'].pct_change().fillna(method='bfill'),
            GS['Close'].pct_change().fillna(method='bfill'),
            JPM['Close'].pct_change().fillna(method='bfill'),
            C['Close'].pct_change().fillna(method='bfill')], axis=1)
rets.columns = ['MS', 'GS', 'JPM', 'C']
rets.corr()

,MS,GS,JPM,C
MS,1.000000,0.844603,0.802317,0.809920
GS,0.844603,1.000000,0.815541,0.791714
JPM,0.802317,0.815541,1.000000,0.852206
C,0.809920,0.791714,0.852206,1.000000


<h3>Select a pair of stocks to trade</h3>
<li>Choose the pair with the highest correlation
<li>You can just eyeball the result and pick the highest correlation pairs
<li>Create a new df 'pairs' with the returns columns of the highest correlation pairs

In [4]:
pairs = pd.concat([JPM['Close'],C['Close']], axis=1)
pairs.columns = ['JPM', 'C']
pairs.head()

,JPM,C
Date,,
2013-01-02,37.123642,38.420940
2013-01-03,37.048836,38.551334
2013-01-04,37.705509,39.520012
2013-01-07,37.747082,39.557259
2013-01-08,37.821896,39.547947


<h3>Calculate the mean and std dev of the columns (returns of the two stocks)</h3>
<li>Note that line 1 contains a NaN. You'll have to get rid of this line
<li>df.describe() returns summary stats for all cols in a dataframe. You can use this to extract means and standard deviations

In [5]:
new_df = rets[['C','JPM']].iloc[1:-1]
c_std = new_df.describe()['C']['std']
j_std = new_df.describe()['JPM']['std']
c_mean = new_df.describe()['C']['mean']
j_mean = new_df.describe()['JPM']['mean']

<h3>Create a new column "Long_JPM_Short_C"</h3>
<li>If the return of C is greater than its mean + 0.25* std
<li> AND
<li>If the return of JPM is less than its mean - 0.25*std
<li>The value in this col should be True or False

In [6]:
rets['Long_JPM_Short_C'] = ((rets['C'] > (c_mean+(0.25*c_std))) & (rets['JPM'] < (j_mean-(0.25*j_std))))

<h3>Create a new column "Long_C_Short_JPM"</h3>
<li>If the return of JPM is greater than its mean + 0.25* std
<li> AND
<li>If the return of C is less than its mean - 0.25*std
<li>The value in this col should be True or False

In [7]:
rets['Long_C_Short_JPM'] = ((rets['JPM'] > (j_mean+(0.25*j_std))) & (rets['C'] < (c_mean-(0.25*c_std))))

<h3>Create a new column "Trade_return"</h3>
<li>If Long_JPM_Short_C is True and Long_C_Short_JPM is False, the value of this column should be the return on JPM minus the return on C
<li>If Long_JPM_Short_C is False and Long_C_Short_JPM is True, the value of this column should be the return on C minus the return on JPM
<li>Otherwise the value should be zero

In [9]:
a = rets[rets['Long_JPM_Short_C'] & ~rets['Long_C_Short_JPM']]
a_true_vals = a['JPM']-a['C']
a_idx = a.index


b = rets[~rets['Long_JPM_Short_C'] & rets['Long_C_Short_JPM']]
b_true_vals = b['C']-b['JPM']
b_idx = b.index

rets['Trade_return'] = 0

rets['Trade_return'].loc[a_idx] = a_true_vals
rets['Trade_return'].loc[b_idx] = b_true_vals

In [10]:
pairs['Trade_return'] = rets['Trade_return']

<h3>Calculate the sum of this new column</h3>
<li>Are you going to get rich?

In [11]:
sum(pairs['Trade_return'])

-0.6496463025381582